# MLflow Tracking

The MLflow Tracking is an API and UI for logging parameters, code versions, metrics, and output files when running your machine learning code and for later visualizing the results.

## Concepts

![Taken from MLflow Docs](https://mlflow.org/docs/latest/_images/tracking-basics.png)

**Runs**

MLflow Tracking is organized around the concept of runs, which are executions of some piece of data science code, for example, a single python train.py execution.


**Experiments** 

An experiment groups together runs for a specific task. 




In [ ]:
import os 
import mlflow 
from mlflow_for_ml_dev.utils.utils import get_root_project

# Create an experiment

In [ ]:
experiment_name = "01-experiment"

In [ ]:
artifact_location = get_root_project() / "experiments" / "mlruns"

mlflow.set_tracking_uri(artifact_location.as_uri())
print(artifact_location)

In [ ]:
# Create experiment
experiment_id = mlflow.create_experiment(name=experiment_name)
print(experiment_id)

In [ ]:
with mlflow.start_run() as run:
    # Log some parameters and metrics here
    print(run.info.run_id)

In [ ]:
# Set the experiment as active experiment
experiment = mlflow.set_experiment(experiment_name)

In [ ]:
with mlflow.start_run() as run:
    # Log some parameters and metrics here
    print(run.info.run_id)

In [ ]:
# demo run
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=10)

with mlflow.start_run(run_name="first-run") as run:
    mlflow.log_param("param1", 5)
    mlflow.sklearn.log_model(sk_model=rfc, artifact_path="sklearn-model")

An experiment can also be created by using `mlflow.set_experiment(experiment_name)`. If the experiment does not exist mlflow creates an experiment using the provided name. Since the `experiment_name` is a path in the workspace is necessary to ensure that the folder structure exists before creating the experiment

## Adding tags

In [ ]:
experiment_name = "01-experiment-2"
experiment_id = mlflow.create_experiment(
    name=experiment_name,
    tags={"topic":"experiment_management", "project_name":"UNKNOWN"}
)

# Set the experiment as active experiment
experiment = mlflow.set_experiment(experiment_name)

In [ ]:
# get the experiment tags
experiment.tags

## Adding a description

In [ ]:
experiment_name = "01-experiment-3"
experiment_id = mlflow.create_experiment(
    name=experiment_name,
    tags={
        "topic":"experiment_management",
        "project_name":"UNKNOWN",
        "mlflow.note.content":"This is a test experiment"})

In [ ]:
# Set the experiment as active experiment
experiment = mlflow.set_experiment(experiment_name)

In [ ]:
# get the experiment tags
experiment.tags

## Update Tags

In [ ]:
experiment_name = "01-experiment-3"
experiment = mlflow.set_experiment(experiment_name)
# experiment tags
experiment.tags

In [ ]:
tags = {
    "tag1": "value1",
    "tag2": "value2"
}
mlflow.set_experiment_tags(tags=tags)


In [ ]:
# get the updated experiment object
experiment = mlflow.set_experiment(experiment_name)

# get the experiment tags
experiment.tags

In [ ]:
# Update Value of tag1
mlflow.set_experiment_tag(key="tag1", value="new_value1")

# get the updated experiment object
experiment = mlflow.set_experiment(experiment_name)

In [ ]:
# get the experiment tags
experiment.tags

## Using the client to set a tag

In [ ]:
client = mlflow.MlflowClient()

In [ ]:
experiment.name

In [ ]:
client.set_experiment_tag(experiment_id = experiment.experiment_id, key="tag3", value="value3")

In [ ]:
experiment = mlflow.set_experiment(experiment_name)

# get the experiment tags
experiment.tags

## Rename Experiment

In [ ]:
new_name = "01-experiment-3-renamed"
client.rename_experiment(experiment_id = experiment.experiment_id, new_name=new_name)

In [ ]:
experiment = mlflow.set_experiment(new_name)

experiment.name

# Avoid errors

In [ ]:
# trying to create an experiment with the same name
mlflow.create_experiment(name="01-experiment-3-renamed")

**To avoid this, we can set the experiment as active experiment or use try-except block**

## Clean Up

In [ ]:
experiments = mlflow.search_experiments(filter_string="name LIKE '01-experiment%'")
# experiments = mlflow.search_experiments()
for experiment in experiments:
    print(f"Deleting: {experiment.name}")
    mlflow.delete_experiment(experiment.experiment_id)
